<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#RTB-Control" data-toc-modified-id="RTB-Control-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RTB Control</a></span><ul class="toc-item"><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Evaluation</a></span></li><li><span><a href="#Ray-Tune" data-toc-modified-id="Ray-Tune-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ray Tune</a></span></li></ul></li></ul></div>

In [1]:
# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# change default style figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

# prevent scientific notations
pd.set_option('display.float_format', lambda x: '%.3f' % x)

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,sklearn,matplotlib

Ethen 2021-08-14 17:09:00 

CPython 3.6.4
IPython 7.15.0

numpy 1.18.5
pandas 1.0.5
sklearn 0.0
matplotlib 3.1.0


In [2]:
!head exp-data/train.txt

0	51	0.000154	1
0	87	0.000140	1
0	33	0.000322	1
0	65	0.000100	1
0	238	0.000169	1
0	65	0.000174	1
0	6	0.000115	1
0	129	0.000046	1
0	57	0.000265	1
0	81	0.000128	1


# RTB Control

- https://github.com/wnzhang/rtbcontrol
- https://resourcium.org/journey/pid-control-brief-introduction

In [3]:
input_path = 'exp-data/train.txt'

df_train = pd.read_csv(input_path, sep='\t', header=None, names=['clicks', 'bids', 'ctrs', 'dummy'])
print(df_train.shape)
df_train.head()

(100000, 4)


,clicks,bids,ctrs,dummy
0,0,51,0.000,1
1,0,87,0.000,1
2,0,33,0.000,1
3,0,65,0.000,1
4,0,238,0.000,1


In [4]:
def compute_enhanced_cpc_bid(base_bid: int, base_ctr: float, ctr: float) -> int:
    return int(base_bid * ctr / base_ctr)

In [5]:
import math


def simulate(
    df: pd.DataFrame,
    target_metric: str,
    reference_point: float,
    control_rounds: int,
    param_p: float,
    param_i: float,
    param_d: float
) -> pd.DataFrame:
    base_ctr = df['clicks'].sum() / df['clicks'].shape[0]
    clicks = df['clicks'].tolist()
    bids = df['bids'].tolist()
    ctrs = df['ctrs'].tolist()
    
    total_cost = 0.0
    total_clicks = 1
    total_wins = 0

    ecpc_records = []
    bids_records = []
    phi_records = []
    win_rate_records = []

    # the i term in pid controller
    error_sum = 0.0

    control_batch_size = len(clicks) // control_rounds
    for i in range(control_rounds):
        # update pid controller
        if i == 0:
            phi = 0.0
        else:
            if target_metric == 'ecpc':
                metric_value = ecpc_records[i - 1]
                metric_value_delta = ecpc_records[i - 2] - ecpc_records[i - 1]
            elif target_metric == 'win_rate':
                metric_value = win_rate_records[i - 1]
                metric_value_delta = win_rate_records[i - 2] - win_rate_records[i - 1]

            error = reference_point - metric_value
            error_sum += error
            phi = param_p * error + param_i * error_sum
            if i > 1:
                phi += param_d * metric_value_delta

        phi = max(phi, min_phi)
        phi = min(phi, max_phi)

        batch_start = i * control_batch_size
        batch_end = (i + 1) * control_batch_size
        for j in range(batch_start, batch_end):
            click = clicks[j]
            bid = bids[j]
            ctr = ctrs[j]

            # update bid
            enhanced_cpc_bid = compute_enhanced_cpc_bid(base_bid, base_ctr, ctr)
            adjusted_bid = enhanced_cpc_bid * math.exp(phi)
            adjusted_bid = max(min_bid, adjusted_bid)

            if adjusted_bid >= bid:
                total_wins += 1
                total_clicks += click
                total_cost += bid

        ecpc = total_cost / total_clicks
        ecpc_records.append(ecpc)

        win_rate = total_wins / batch_end
        win_rate_records.append(win_rate)

        phi_records.append(phi)
        bids_records.append(adjusted_bid)

    df_records = pd.DataFrame({
        'bids': bids_records,
        'phi': phi_records,
        'win_rate': win_rate_records,
        'ecpc': ecpc_records
    })
    return df_records

In [6]:
# hyper parameters
target_metric = 'ecpc'
reference_point = 40000.0
# target_metric = 'win_rate'
# reference_point = 0.15

control_rounds = 40

base_bid = 69
min_bid = 5

# part of pid controller
param_p = 0.0005
param_i = 0.000001
param_d = 0.0001
min_phi = -2
max_phi = 5


df_records = simulate(
    df_train,
    target_metric,
    reference_point,
    control_rounds,
    param_p,
    param_i,
    param_d
)
df_records.head(50)

,bids,phi,win_rate,ecpc
0,15.000,0.000,0.160,3249.333
1,445.239,5.000,0.571,54562.000
2,11.639,-2.000,0.386,54733.333
3,5.000,-2.000,0.294,54834.667
4,5.000,-2.000,0.238,55035.333
5,5.000,-2.000,0.207,41475.750
6,5.000,0.594,0.220,47222.500
7,5.000,-2.000,0.196,47353.500
8,5.000,-2.000,0.176,47427.750
9,5.000,-2.000,0.162,38043.200


In [7]:
hi

NameError: name 'hi' is not defined

In [ ]:
def plot_records(df_records: pd.DataFrame, target_metric: str, reference_point: float):
    fig, (ax1, ax2) = plt.subplots(1, 2)

    ax1.plot(df_records[target_metric])
    ax1.set_xlabel('iterations')
    ax1.set_ylabel(target_metric)
    ax1.axhline(y=reference_point, color='r', linestyle='-')

    ax2.plot(df_records['bids'])
    ax2.set_xlabel('iterations')
    ax2.set_ylabel('bids')

    plt.tight_layout()
    plt.show()

In [ ]:
# change default style figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

plt.plot(df_records[target_metric])
plt.show()

In [ ]:
plt.plot(df_records['bids'])
plt.show()

## Evaluation

In [ ]:
def compute_settling_time(metrics, reference_point, error_band=0.1):
    settled = False
    settling_time = 0
    for i, value in enumerate(metrics):
        error = reference_point - value
        if abs(error) / reference_point <= error_band and not settled:
            settled = True
            settling_time = i
        elif abs(error) / reference_point > error_band:
            settled = False
            settling_time = len(metrics)

    return settling_time

In [ ]:
def compute_rise_time(metrics, reference_point, error_band=0.1):
    rise_time = 0
    for i, value in enumerate(metrics):
        error = reference_point - value
        if abs(error) / reference_point <= error_band:
            rise_time = i
            break

    return rise_time

In [ ]:
def compute_rmse_ss(metrics, reference_point):
    settled = False
    settling_time = compute_settling_time(metrics, reference_point)
    rmse = 0.0
    rounds = len(metrics)
    if settling_time >= rounds:
        settling_time = rounds - 1
    for i in range(settling_time, rounds):
        rmse += (metrics[i] - reference_point) * (metrics[i] - reference_point)
    rmse /= (rounds - settling_time)
    rmse = math.sqrt(rmse) / reference_point
    return rmse

In [ ]:
compute_settling_time(df_records[target_metric], reference_point)

In [ ]:
compute_rise_time(df_records[target_metric], reference_point)

In [ ]:
compute_rmse_ss(df_records[target_metric], reference_point)

## Ray Tune

In [ ]:
from ray import tune


def training_function(
    config,
    checkpoint_dir=None,
    df=None,
    target_metric='ecpc',
    reference_point=40000,
    control_rounds=40
):
    """
    https://docs.ray.io/en/master/tune/api_docs/trainable.html#tune-with-parameters
    """
    param_p, param_i, param_d = config["param_p"], config["param_i"], config["param_d"]
    df_records = simulate(
        df,
        target_metric,
        reference_point,
        control_rounds,
        param_p,
        param_i,
        param_d
    )
    score = compute_settling_time(df_records[target_metric], reference_point)
    tune.report(score=score)

In [ ]:
target_metric = 'win_rate'
reference_point = 0.3
run = tune.with_parameters(
    training_function,
    df=df_train,
    target_metric=target_metric,
    reference_point=reference_point,
    control_rounds=control_rounds
)

analysis = tune.run(
    run,
    config={
        "param_p": tune.grid_search([0.0005, 0.001, 0.01, 0.1, 0.5]),
        "param_i": tune.choice([0.0005, 0.001, 0.01, 0.1, 0.5]),
        "param_d": tune.grid_search([0.0005, 0.001, 0.01, 0.1, 0.5])
    }
)

# Get a dataframe for analyzing trial results.
df_analysis_results = analysis.results_df
df_analysis_results

In [ ]:
best_config = analysis.get_best_config(metric="score", mode="min")
print(best_config)

In [ ]:
df_records = simulate(
    df_train,
    target_metric,
    reference_point,
    control_rounds,
    **best_config
)
df_records

In [ ]:
def plot_records(df_records: pd.DataFrame):
    fig, (ax1, ax2) = plt.subplots(1, 2)

    ax1.plot(df_records[target_metric])
    ax1.set_xlabel('iterations')
    ax1.set_ylabel(target_metric)

    ax2.plot(df_records['bids'])
    ax2.set_xlabel('iterations')
    ax2.set_ylabel('bids')

    plt.tight_layout()
    plt.show()
    
    
    
plot_records(df_records)

In [ ]:
input_path = 'exp-data/test.txt'
df_test = pd.read_csv(input_path, sep='\t', header=None, names=['clicks', 'bids', 'ctrs', 'dummy'])
print(df_test.shape)
df_test.head()

In [ ]:
df_records = simulate(
    df_test,
    target_metric,
    reference_point,
    control_rounds,
    **best_config
)
df_records

In [ ]:
compute_settling_time(df_records[target_metric], reference_point)

In [ ]:
plt.plot(df_records[target_metric])
plt.show()

In [ ]:
plt.plot(df_records['bids'])
plt.show()